In [1]:
import json
import pandas as pd

# Load in the JSON file
with open("/home/msaad/workspace/honors-thesis/data-collection/data/categorized_data.json", "r") as f:
    data = json.load(f)

# Load in cleaned csv
df = pd.read_csv("/home/msaad/workspace/honors-thesis/data-collection/data/full_cleaned_data.csv")

In [2]:
data.keys()

dict_keys(['about', 'academics', 'admissions', 'admissions-aid', 'graduate', 'library', 'life', 'live', 'research-foundation', 'scholarships-aid', 'support'])

In [3]:
url_list = df['url'].to_list()

url_shortened_list = [url.split("brockport.edu")[1] for url in url_list]

# Pull out all the portions the URL goes through
split_urls = [[portion for portion in url.split("/") if portion != ''] for url in url_shortened_list]
first_portions = [portion[0] for portion in split_urls if len(portion) > 0]

In [4]:
pd.Series(first_portions).value_counts()

academics              1225
support                 485
live                    278
life                    274
about                   139
graduate                 50
admissions               48
admissions-aid           29
library                  20
research-foundation      17
scholarships-aid         12
Name: count, dtype: int64

# Try and build the search engine

In [77]:
import os

def save_to_txt(data: str, filename: str, category: str, subcategory: str = None):
    base_file_path = "/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data"
    
    filename = filename.removeprefix("https://www2.brockport.edu/").replace("/", "-")

    path = f"{base_file_path}/{category}{(f'/{subcategory}' if subcategory != None else '')}/"

    print(path)  

    if not os.path.exists(path):
        os.makedirs(path)

    with open(f"{path}{filename}.txt", "w") as f:
        f.write(data)

In [78]:
# Pull high level categories (i.e. about, academics, admissions, etc.)
for category, content in data.items():

    # Pull first level of category -- the "subcategory". 
    # This can point to either another category or data directly
    for subcategory, inner_content in content.items():

        # If its pointing to data directly, save it off
        if isinstance(inner_content, str):
            save_to_txt(inner_content, subcategory, category)
            continue
        
        # If it points to another category, pull the data from that category
        elif isinstance(inner_content, dict):
            for url, text in inner_content.items():
                save_to_txt(text, url, category, subcategory)
                continue

/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/
/home/msaad/workspace/honors-thesis/data-collectio

# Lets see how we can load these using langchain now

In [92]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

path = "/home/msaad/workspace/honors-thesis/data-collection/data/categorized_datastore/raw_data/about/"

loader = DirectoryLoader(path, glob="*.txt", loader_cls=TextLoader)

docs = loader.load()

len(docs)

29

In [95]:
os.listdir(path)

['about-archives.txt',
 'about-sustainability-how-you-can-be-sustainable.txt',
 'about-economic-impact.txt',
 'about-accountability-directory.txt',
 'about-town-gown-quality-life.txt',
 'president',
 'diversity',
 'about-sustainability-electronic-recycling.txt',
 'about-brockport-downtown-students.txt',
 'about-facts.txt',
 'about-sustainability-waste-audit.txt',
 'about-brockport-downtown.txt',
 'about-brockport-downtown-faculty-staff.txt',
 'about-facts-academics.txt',
 'about-accountability.txt',
 'about-town-gown-communications-outreach.txt',
 'about-privacy.txt',
 'title-ix',
 'about-brockport-downtown-parking.txt',
 'strategic-plan',
 'about-town-gown-alcohol-drugs.txt',
 'about-town-gown.txt',
 'about-sustainability-campus-race-to-zero-waste.txt',
 'about-sustainability-stormwater.txt',
 'about-accountability-committees.txt',
 'about-brockport-downtown-event-services.txt',
 'about-sustainability.txt',
 'middlestates',
 'about-accountability-academic-assessment.txt',
 'about-loca